In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

import os, json, math, librosa

import IPython.display as ipd
import librosa.display

import tensorflow as tf
import tensorflow.keras as keras

from tensorflow.keras import Sequential
from tensorflow.keras.layers import Conv2D

import sklearn.model_selection as sk

from sklearn.model_selection import train_test_split


ModuleNotFoundError: No module named 'librosa'

In [ ]:
MUSIC = '/Users/apoorva/Desktop/CODES/Data/genres_original/'
music_dataset = [] 
genre_target = [] 
for root, dirs, files in os.walk(MUSIC):
    for name in files:
        filename = os.path.join(root, name)
        if filename != '/data/genres_original/jazz/jazz.00054.wav':
            music_dataset.append(filename)
            genre_target.append(filename.split("/")[5])
            

In [ ]:
audio_path = music_dataset[500]
ipd.Audio(audio_path)

In [ ]:
DATASET_PATH = '/Users/apoorva/Desktop/CODES/Data/genres_original/'
JSON_PATH = "music_genre.json"
SAMPLE_RATE = 22050
TRACK_DURATION = 30 
SAMPLES_PER_TRACK = SAMPLE_RATE * TRACK_DURATION


def save_mfcc(dataset_path, json_path, num_mfcc=13, n_fft=2048, hop_length=512, num_segments=5):

    data = {
        "mapping": [],
        "labels": [],
        "mfcc": []
    }

    samples_per_segment = int(SAMPLES_PER_TRACK / num_segments)
    num_mfcc_vectors_per_segment = math.ceil(samples_per_segment / hop_length)

    
    for i, (dirpath, dirnames, filenames) in enumerate(os.walk(dataset_path)):

        if dirpath is not dataset_path:

            semantic_label = dirpath.split("/")[-1]
            data["mapping"].append(semantic_label)
            print("\nProcessing: {}".format(semantic_label))

            for f in filenames:

                file_path = os.path.join(dirpath, f)
            
                if file_path != '/Users/apoorva/Desktop/CODES/Data/genres_original/jazz/jazz.00054.wav':

                    signal, sample_rate = librosa.load(file_path, sr=SAMPLE_RATE)
                
                
                    for d in range(num_segments):

                        start = samples_per_segment * d
                        finish = start + samples_per_segment

                        mfcc = librosa.feature.mfcc(signal[start:finish], sample_rate, n_mfcc=num_mfcc, n_fft=n_fft, hop_length=hop_length)
                        mfcc = mfcc.T

                        if len(mfcc) == num_mfcc_vectors_per_segment:
                            data["mfcc"].append(mfcc.tolist())
                            data["labels"].append(i-1)
                            print("{}, segment:{}".format(file_path, d+1))

    with open(json_path, "w") as fp:
        json.dump(data, fp, indent=4)

In [ ]:
save_mfcc(DATASET_PATH, JSON_PATH, num_segments=6)

In [ ]:
DATA_PATH = "./music_genre.json"


def load_data(data_path):
    
    with open(data_path, "r") as fp:
        data = json.load(fp)

    X = np.array(data["mfcc"])
    y = np.array(data["labels"])
    z = np.array(data['mapping'])
    return X, y,z


In [ ]:
def prepare_datasets(test_size, validation_size):
    
    # load data
    X, y,z = load_data(DATA_PATH)

    # create train, validation and test split
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size)
    X_train, X_validation, y_train, y_validation = train_test_split(X_train, y_train, test_size=validation_size)

    # add an axis to input sets
    X_train = X_train[..., np.newaxis]
    X_validation = X_validation[..., np.newaxis]
    X_test = X_test[..., np.newaxis]

    return X_train, X_validation, X_test, y_train, y_validation, y_test,z

In [ ]:
def build_model(input_shape):

    # build network topology
    model = keras.Sequential()

    # 1st conv layer
    model.add(keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=input_shape))
    model.add(keras.layers.MaxPooling2D((3, 3), strides=(2, 2), padding='same'))
    model.add(keras.layers.BatchNormalization())

    # 2nd conv layer
    model.add(keras.layers.Conv2D(32, (3, 3), activation='relu'))
    model.add(keras.layers.MaxPooling2D((3, 3), strides=(2, 2), padding='same'))
    model.add(keras.layers.BatchNormalization())

    # 3rd conv layer
    model.add(keras.layers.Conv2D(32, (2, 2), activation='relu'))
    model.add(keras.layers.MaxPooling2D((2, 2), strides=(2, 2), padding='same'))
    model.add(keras.layers.BatchNormalization())

    # flatten output and feed it into dense layer
    model.add(keras.layers.Flatten())
    model.add(keras.layers.Dense(64, activation='relu'))
    model.add(keras.layers.Dropout(0.8))

    # output layer
    model.add(keras.layers.Dense(10, activation='softmax'))

    return model

In [ ]:
def predict(model, X, y):

    X = X[np.newaxis, ...] 

    prediction = model.predict(X)

    predicted_index = np.argmax(prediction, axis=1)
    
    target = z[y]
    predicted = z[predicted_index]

    print("Target: {}, Predicted label: {}".format(target, predicted))


In [ ]:
X_train, X_validation, X_test, y_train, y_validation, y_test,z = prepare_datasets(0.25, 0.2)

In [ ]:
input_shape = (X_train.shape[1], X_train.shape[2], 1)
model = build_model(input_shape)

In [ ]:
optimiser = keras.optimizers.Adam(learning_rate=0.0001)

In [ ]:
model.compile(optimizer=optimiser,
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])


In [ ]:
model.summary()

In [ ]:
history = model.fit(X_train, y_train, validation_data=(X_validation, y_validation), batch_size=32, epochs=30)

Training few more epochs

In [ ]:
model.fit(X_train, y_train, validation_data=(X_validation, y_validation), batch_size=32, epochs=30)

In [ ]:
model.fit(X_train, y_train, validation_data=(X_validation, y_validation), batch_size=32, epochs=30)

In [ ]:
model.fit(X_train, y_train, validation_data=(X_validation, y_validation), batch_size=32, epochs=30)

In [ ]:
model.fit(X_train, y_train, validation_data=(X_validation, y_validation), batch_size=32, epochs=30)

In [ ]:
model.fit(X_train, y_train, validation_data=(X_validation, y_validation), batch_size=32, epochs=30)

In [ ]:
model.fit(X_train, y_train, validation_data=(X_validation, y_validation), batch_size=32, epochs=30)


In [ ]:
model.fit(X_train, y_train, validation_data=(X_validation, y_validation), batch_size=32, epochs=10)

In [ ]:
model.fit(X_train, y_train, validation_data=(X_validation, y_validation), batch_size=32, epochs=10)

In [ ]:
model.fit(X_train, y_train, validation_data=(X_validation, y_validation), batch_size=32, epochs=10)

In [ ]:
model.fit(X_train, y_train, validation_data=(X_validation, y_validation), batch_size=32, epochs=10)

In [ ]:
model.fit(X_train, y_train, validation_data=(X_validation, y_validation), batch_size=32, epochs=100)

In [ ]:
model_json = model.to_json()
with open("music_genre(80).json", "w") as json_file:
    json_file.write(model_json)

In [ ]:
model.save_weights("music_genre(80).h5")
print("Saved model to disk")

In [ ]:
# pick a sample to predict from the test set
X_to_predict = X_test[500]
y_to_predict = y_test[500]

# predict sample
predict(model, X_to_predict, y_to_predict)

In [ ]:
from keras.models import model_from_json

In [ ]:
json_file = open('music_genre(80).json', 'r')
model = json_file.read()
json_file.close()
loaded_model = model_from_json(model)

In [ ]:
loaded_model.load_weights("music_genre(80).h5")
print("Loaded model from disk")

In [ ]:
optimiser = keras.optimizers.Adam(learning_rate=0.0001)
loaded_model.compile(optimizer=optimiser,
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
loaded_model

In [ ]:

tf.keras.models.save_model(loaded_model,'music_model.hdf5')


In [ ]:
#plot_history(history)

# evaluate model on test set
test_loss, test_acc = loaded_model.evaluate(X_test, y_test, verbose=2)
train_loss, train_acc = loaded_model.evaluate(X_train,y_train,verbose=2)
print('\nTest accuracy:', test_acc)
print('\nTrain accuracy:', train_acc)